# Exploring Gender Biases in Word2vec <br> 

Write a short intro here 


In [2]:
#Imports
import torch
from torch import nn, optim, sigmoid, softmax
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchtext.data.utils import get_tokenizer
import string

## 1- Define Word2Vec Model 

In [ ]:
# NOTES:
# inspiration: https://github.com/OlgaChernytska/word2vec-pytorch
# We are using the skip gram method because it works better on smaller data sets (which we are limited to), source -  https://arxiv.org/pdf/1301.3781.pdf 
# Skim-gram predicts a context word from a given middle word in the sentance.
# Use adam optimizer (better than SGD) - source: _______
# Applying regularization to our embedding layer: clamping each weiaght vector of each neuron to |w| < 1 ( via max_norm ==1) - prevent exploding gradients
  #the norm restrictions also helps push cosine similarity between words (which we will use to analyze bias)



**Create Tokens** <br>

Justifications:

- punctuation: remove - source : ______________
- lowercase words - because we are only looking at biasis between male/female

In [40]:
def create_tokens(data, min_freq):
    """
    create_vocabulary finds the number of occurences of each word, creates vocuabulary with words with > N_freq, and lebels them with ids

    :param data: list of strings
    :return: a list of tokens 
    """ 
    tokens = []
    for sentance in data:
      #split into tokens, convert to lowercase 
      sentance = sentance.translate(str.maketrans('', '', string.punctuation)) #remove punctuations
      sentance = sentance.translate(str.maketrans('', '', string.digits)) #remove digits
      tokenizer = get_tokenizer("basic_english", language="en") #remove unessasary characters, splits into spaces
      tokens.append(tokenizer(sentance))
  
    return tokens
  

**Define word 2 vec model** <br>

- choosing the top N most frequent words - why? so its easier to train - and not wasting time to train words that appear once in a text sequence. 

In [42]:
from gensim.models import Word2Vec
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
model = Word2Vec(sentences, min_count=1)

In [46]:
from gensim.models import Word2Vec

min_freq = 50 #hyper parameter

sentences = create_tokens(["hey my name is", "you are coool"], min_freq)
print(sentences)
model = Word2Vec(sentences, min_count=1)

[['hey', 'my', 'name', 'is'], ['you', 'are', 'coool']]


## 2 - Create Data sets with wiki and ArXiv from the Pile

In [ ]:
# Credit: https://github.com/noanabeshima/wikipedia-downloader
!pip install tensorflow==2.2.0
#2.8.0
#2.2.0
!pip install tensorflow-datasets=4.3
# !pip install tfds-nightly==3.1.0.dev202007060105
!pip install fire==0.3.1
!pip install tqdm==4.47.0
!pip install joblib==0.15.1
!pip install apache-beam==2.22.0


     |████████████████████████████████| 516.2 MB 4.2 kB/s 
     |████████████████████████████████| 3.0 MB 63.5 MB/s 
     |████████████████████████████████| 454 kB 59.5 MB/s 
     |████████████████████████████████| 2.9 MB 44.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalli

     |████████████████████████████████| 298 kB 8.4 MB/s 
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
     |████████████████████████████████| 3.4 MB 6.8 MB/s 
     |████████████████████████████████| 151 kB 67.2 MB/s 
     |████████████████████████████████| 1.4 MB 69.1 MB/s 
     |████████████████████████████████| 63.8 MB 1.2 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
     |████████████████████████████████| 829 kB 50.6 MB/s 
     |████████████████████████████████| 508 kB 53.5 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 113 kB 54.8 MB/s 
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43513 sha256=3c28ec9d9d592f81a504e3866a2195bbaaa32440cea6ad4f85d80948b02e9ee1
  Stored in directory: /root/.cache/pip/wheels/bc/49/5f/fdb5b9d85055c478213e0158ac122b596816149a02d82e0ab1
  C

In [ ]:
import os
# import json
# import tensorflow as tf
import tensorflow_datasets as tfds
# from tqdm import tqdm
# from joblib import Parallel, delayed
import fire
# !pip install git+https://github.com/cehorn/GLRM.git
# https://github.com/google/python-fire

def process_article(article):
    # Converts an article to a single text file
    title = article['title'].numpy().decode('UTF-8')
    text = article['text'].numpy().decode('UTF-8')
    return title+"\n\n"+text

def main(n_jobs: int = 1):
    # Downloads wikipedia dataset using tensorflow_datasets into 10 json files
    try:
        os.mkdir('output')
    except:
        pass

    for interval in range(10):
        if f'wikipedia-en-{interval}.json' not in os.listdir('./output'):
            ds = tfds.load('wikipedia/20200301.en', split=f'train[{str(interval)}0%:{str(interval+1)}0%]')

            result = Parallel(n_jobs=n_jobs)(delayed(process_article)(article) for article in tqdm(ds))

            result = json.dumps(result)

            file = open(f"output/wikipedia-en-{interval}.json", "w")
            file.write(result)
            file.close()

if __name__ == '__main__':
    fire.Fire(main)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/258 [00:00<?, ? file/s]

## 3 - Debiasing Techniques


### 3.1 - Zhao et al.'s Method Using an Objective Function with Catigorized Words


### 3.2 - Bolukbasi et al.'s Soft-debaising Method

### 3.3 - Savani et al.'s First Order Optimization

## 4 - Measuring Bias

### 4.1 - Direct Bias

### 4.2 - Indirect Bias


### 4.3 - WEAT Metric

## 5 - Comparison and Conclusions